# Prepare datasets

The file HistoricalResultsByCount\ \[AKAN-1901-2018\].csv contains several tables, which I prefer to split up into individual csv's.

## Get latlong.csv

The first table given is the latitude and longitude of AKAN.  


## Do the same for remaining 4 tables

They are data for Weather, Effort, Sponsoring Organization, and "Checklist", which is the bird species reported.


## Weather table

It is 50 rows.



## Effort table

The effort table has incorrect column names in the csv; they should be ``"CountYear5,CountDate,NumParticipants,NumHours,NumSpeciesReported"``

## Sponsoring organization

Again, the columns are named incorrectly; they should be "CountYear, SponsoringOrganization".



# Checklist

* While competing bird species count is probably relevant, we should first isolate data for Black-Capped Chickadee and Bald Eagle alone.  

* The format of this part of the csv is strange; it is better elucidated by looking at the .xlsx output of the CBC's documentation: http://docs.audubon.org/sites/default/files/documents/cbc_historical_results_download_file_format.pdf

* This process can then be reused later to isolate any species data

### Get chickadee data only

The contents does not look ready for import into pandas.  Furthermore, it is unclear what is meant by these numbers.

# CBC Historical Results Download File Format

From http://docs.audubon.org/sites/default/files/documents/cbc_historical_results_download_file_format.pdf:

Also by [examining the excel file](excel_scrot.png), we can determine the meaning out the output from the above commands.

For example: ``Total Hrs.: 253.00",1908,7.5415,`` indicated there were 1908 actual observations of the birds in 253 hours.

It is still unclear if the observations were of unique birds or not.


### Prep txt files for chickadee and bald eagle counts only

## Prepare csv format for observations

``countYear, month, date, year, NumHrs, Ncount, density``

The following cells create data looking like:

``42,12,27,1941, ,10,0.0000``

``61,1,2,1961, 29.00,52,1.7931``

``63,12,23,1962, 33.00,23,0.6970``

``...``


### Bald Eagle


In [85]:
import sys

f = open('bald_eagle_only.txt','r').readlines()
countYear = []
month = []
date = []
year = []
NumHrs = []
Ncount = []
density = []
for line in f:
    if 'Date' in line:
        year +=  [line.strip().split('/')[-1]]
        date +=  [line.strip().split('/')[-2]]
        month += [line.strip().split('/')[-3].split(':')[-1].strip()]
        
    if 'Ha' in line:
        countYear += [line.strip().split()[-1].replace('[','').replace(']','')]
        
    if 'Total' in line:
        NumHrs += [line.strip().split(',')[0].split(':')[-1].replace('"','')]
        Ncount += [line.strip().split(',')[1:][0]]
        density+= [line.strip().split(',')[1:][1]]

f.close()
################################
#  write it all to csv format  #
################################

#with open('bald_eagle_counts.csv', 'w') as f:
#    for i in range(len(countYear)):
#        f.write('{},{},{},{},{},{},{}\n'.format(countYear[i],month[i],date[i],year[i],NumHrs[i],Ncount[i],density[i]))
#f.close()
    
    

In [84]:
import sys

f = open('chickadee_only.txt','r').readlines()
countYear = []
month = []
date = []
year = []
NumHrs = []
Ncount = []
density = []
for line in f:
    if 'Date' in line:
        year +=  [line.strip().split('/')[-1]]
        date +=  [line.strip().split('/')[-2]]
        month += [line.strip().split('/')[-3].split(':')[-1].strip()]
        
    if 'Poe' in line:
        countYear += [line.strip().split()[-1].replace('[','').replace(']','')]
        
    if 'Total' in line:
        NumHrs += [line.strip().split(',')[0].split(':')[-1].replace('"','')]
        Ncount += [line.strip().split(',')[1:][0]]
        density+= [line.strip().split(',')[1:][1]]

f.close()

################################
#  write it all to csv format  #
################################
#
#with open('chickadee_counts.csv', 'w') as f:
#    for i in range(len(countYear)):
#        f.write('{},{},{},{},{},{},{}\n'.format(countYear[i],month[i],date[i],year[i],NumHrs[i],Ncount[i],density[i]))
#f.close()
    

### Concatenate headers with csv data

``> echo "countYear, month, date, year, NumHrs, Ncount, density"  > header.csv``

``> echo $(cat header.csv bald_eagle_counts.csv) > bald_eagle_counts.csv``

``> echo $(cat header.csv bald_eagle_counts.csv) > chickadee_counts.csv``

``> head -n5 chickadee_counts.csv``

``    countYear, month, date, year, NumHrs, Ncount, density``

``    42,12,27,1941, ,10,0.0000``

``    61,1,2,1961, 29.00,52,1.7931``

``    63,12,23,1962, 33.00,23,0.6970``

``    64,12,22,1963, 21.00,6,0.2857``


In [28]:
import numpy as np
import pandas as pd

effortdf = pd.DataFrame.from_csv('effort.csv')
weatherdf = pd.DataFrame.from_csv('weather.csv')
droughtdf = pd.DataFrame.from_csv('drought_fig-1.csv')
heavyPrecipdf = pd.DataFrame.from_csv('heavy-precip_fig-1.csv')
precipdf = pd.DataFrame.from_csv('precipitation_fig-1.csv')
temperaturedf = pd.DataFrame.from_csv('temperature_fig-1.csv')
bald_eagledf = pd.DataFrame.from_csv('bald_eagle_counts.csv')
chickadeedf = pd.DataFrame.from_csv('chickadee_counts.csv')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: from_csv is deprecated. Please use read_csv(

### Need to get all tables indexed by count year

In [42]:
count_year_dict     = dict()
inv_count_year_dict = dict()

for i in range(-5,120):
    count_year_dict[i] = 1900+i-1
    inv_count_year_dict[1900+i-1] = i
    
for i in range(100,119):
    print(i, count_year_dict[i], inv_count_year_dict[count_year_dict[i]])
    

100 1999 100
101 2000 101
102 2001 102
103 2002 103
104 2003 104
105 2004 105
106 2005 106
107 2006 107
108 2007 108
109 2008 109
110 2009 110
111 2010 111
112 2011 112
113 2012 113
114 2013 114
115 2014 115
116 2015 116
117 2016 117
118 2017 118


In [43]:
count_year_dict.keys()

dict_keys([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119])

In [44]:
inv_count_year_dict.keys()

dict_keys([1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [45]:
inv_count_year_dict[2009]

110